In [ ]:
###Поиск признаков, которые могут обобщаться на многоязычные данные (минимум 10 признаков рассмотрено,, обучать можно любую модель, оценивать нужно на validation.csv)  - 3 балла

In [ ]:
Некоторые идеи позаимстованы у Сергея Косяка

In [34]:
import pandas as pd
import re
import sklearn

In [18]:
train1 = pd.read_csv('toxic-comment-train.csv')
validation = pd.read_csv('validation_with_translation.csv')

In [ ]:
Отфильтруем в валидации только те фрагменты, где перевод выполнен, чтобы сопоставление было корректным (некоторые тексты не перевелись)

In [ ]:
Признаки, которые нужно считать на оригинале, даже если используется перевод (либо предположительно не зависят от языка)

In [20]:
def has_link_or_email(text): #на основе кода Сергея Косяка, но он не подсчитывал емейлы
    if len(re.findall("https?://[^ ]+", text)) + len(re.findall('[^ ]@', text)) > 0: #после непробела, чтобы не считать тэги пользователей соцсети
        return True
    else:
        return False

In [21]:
def punctuation_blocs(text): #количество блоков пунктуации, несколько знаков препинания подряд считаются за один
    return len(re.findall('[\.,\?!—]+', text))

In [22]:
def expressive_punctuation(text): #есть ли несколько вопросительных/восклицательных знаков подряд
    return len(re.findall('[\?!]{2,}', text))

In [32]:
punctuation = '!"(),-.:;?—«»'

In [24]:
def longest_same_nonpunctuation_char(text): #на основе кода Сергея Косяка, но он и пунктуацию подсчитывал
    length = 0
    curr_length = 1
    for i in range(1,len(text)):
        if text[i] == text[i-1]:
            if text[i] not in punctuation:
                curr_length += 1
            else:
                if curr_length > length:
                    length = curr_length
                    curr_length = 1
    return length

In [25]:
def digit_sequences(text):
    return len(re.findall('[0-9]+', text))

In [26]:
def sentenize(text): #код Сергея Косяка
    sents = re.split("[\n.!?]( |$)", text)
    return [s for s in sents if len(s) > 0 and s != " "]

In [38]:
def caps_ratio_not_sent_begining(text): #на основе кода Сергея Косяка, 
    sents = sentenize(text) #но я не учитываю заглавные буквы в начале предложений
    non_begin_sym = len(text) - len(sents)
    caps = 0
    for sent in sents:
        if len(sent) > 1:
            for letter in sent[1:]:
                if letter not in punctuation and letter != " ":
                    if letter.lower() != letter:
                        caps += 1
    if non_begin_sym:
        return caps/non_begin_sym
    else:
        return 0

In [30]:
def splitted_into_paragraphs(text):
    paragraphs = text.split('\n')
    if len(paragraphs) > 1:
        return True
    else:
        return False

Признаки, которые могут различаться в оригинале и в английском переводе

In [31]:
#Для подсчёта длины предложения в словах и средней длины слова нужна токенизация
def tokenize(text): #код Сергея Косяка
    text = re.sub(f"[{punctuation}]", " ", text)
    text = re.sub("\s+", " ", text)
    return text.split()

In [41]:
def mean_sent_len(text):
    return len(tokenize(text))/len(sentenize(text))

In [46]:
def duplicates(tokens): #доля повторяющихся слов
    if tokens:
        return (len(tokens) - len(set(tokens)))/len(tokens)
    else:
        return 0

In [48]:
from statistics import mean
def mean_word_len(tokens):
    if tokens:
        return mean([len(token) for token in tokens])
    else:
        return 0

In [ ]:
Оформим тренировочный датасет

In [ ]:
train1['tokens'] = train1['comment_text'].apply(tokenize)

train1['has_link_or_em'] = train1['comment_text'].apply(has_link_or_email)
train1['punctuation_blocs'] = train1['comment_text'].apply(punctuation_blocs)
train1['expressive_punctuation'] = train1['comment_text'].apply(expressive_punctuation)
train1['longest_same_nonpunctuation_char'] = train1['comment_text'].apply(longest_same_nonpunctuation_char)
train1['digit_sequences'] = train1['comment_text'].apply(digit_sequences)
train1['capitalization'] = train1['comment_text'].apply(caps_ratio_not_sent_begining)
train1['splitted_into_paragraphs'] = train1['comment_text'].apply(splitted_into_paragraphs)
train1['mean_sent_len'] = train1['comment_text'].apply(mean_sent_len)
train1['duplicates'] = train1['tokens'].apply(duplicates)
train1['mean_word_len'] = train1['tokens'].apply(mean_word_len)

Обучим модель. Цель - посмотреть, как соотносится результат на английском и на иноязычеых данных, а не достичь высокого качества, так что параметры по умолчанию подойдут

In [50]:
X = train1[['has_link_or_em', 'punctuation_blocs', 'expressive_punctuation', 'longest_same_nonpunctuation_char', 'digit_sequences', 'capitalization', 'splitted_into_paragraphs', 'mean_sent_len', 'duplicates', 'mean_word_len']]
y = train1['toxic']

In [54]:
from sklearn.linear_model import LogisticRegression

In [55]:
model = LogisticRegression()

In [56]:
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y)

In [57]:
model.fit(X_train, y_train)

C:\Users\lizan\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [58]:
prediction = model.predict(X_test)

In [62]:
sklearn.metrics.f1_score(y_test, prediction)

0.11198361215431887

In [64]:
sklearn.metrics.roc_auc_score(y_test, prediction)

0.5287304275691078

In [67]:
from sklearn.tree import DecisionTreeClassifier

In [68]:
model2 = DecisionTreeClassifier()

In [69]:
model2.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [70]:
prediction2 = model2.predict(X_test)

In [71]:
sklearn.metrics.f1_score(y_test, prediction2)

0.21070327822919585

In [72]:
sklearn.metrics.roc_auc_score(y_test, prediction2)

0.5639578873894445

Вообще-то это работает ужасно, но посмотрим, станет ли ещё хуже на многоязычных данных

Попробуем везде использовать исходный текст

In [78]:
really_translated['tokens'] = really_translated['comment_text'].apply(tokenize)

really_translated['has_link_or_em'] = really_translated['comment_text'].apply(has_link_or_email)
really_translated['punctuation_blocs'] = really_translated['comment_text'].apply(punctuation_blocs)
really_translated['expressive_punctuation'] = really_translated['comment_text'].apply(expressive_punctuation)
really_translated['longest_same_nonpunctuation_char'] = really_translated['comment_text'].apply(longest_same_nonpunctuation_char)
really_translated['digit_sequences'] = really_translated['comment_text'].apply(digit_sequences)
really_translated['capitalization'] = really_translated['comment_text'].apply(caps_ratio_not_sent_begining)
really_translated['splitted_into_paragraphs'] = really_translated['comment_text'].apply(splitted_into_paragraphs)
really_translated['mean_sent_len'] = really_translated['comment_text'].apply(mean_sent_len)
really_translated['duplicates'] = really_translated['tokens'].apply(duplicates)
really_translated['mean_word_len'] = really_translated['tokens'].apply(mean_word_len)

C:\Users\lizan\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\lizan\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\lizan\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

In [84]:
X_valid_original = really_translated[['has_link_or_em', 'punctuation_blocs', 'expressive_punctuation', 'longest_same_nonpunctuation_char', 'digit_sequences', 'capitalization', 'splitted_into_paragraphs', 'mean_sent_len', 'duplicates', 'mean_word_len']]
y_valid = really_translated['toxic']

In [82]:
prediction_original1 = model.predict(X_valid_original)

In [85]:
sklearn.metrics.f1_score(y_valid, prediction_original1)

0.08908685968819599

In [87]:
sklearn.metrics.roc_auc_score(y_valid, prediction_original1)

0.5205767114241691

In [88]:
prediction_original2 = model2.predict(X_valid_original)

In [89]:
sklearn.metrics.f1_score(y_valid, prediction_original2)

0.20265548567435357

In [90]:
sklearn.metrics.roc_auc_score(y_valid, prediction_original2)

0.5354996698217037

In [ ]:
На оригинале метрики ниже, чем на английских текстах

Используем перевод там, где предполагается различие

In [91]:
really_translated['tokens'] = really_translated['translation'].apply(tokenize)

really_translated['mean_sent_len'] = really_translated['translation'].apply(mean_sent_len)
really_translated['duplicates'] = really_translated['tokens'].apply(duplicates)
really_translated['mean_word_len'] = really_translated['tokens'].apply(mean_word_len)

C:\Users\lizan\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\lizan\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\lizan\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

In [94]:
X_valid_translated = really_translated[['has_link_or_em', 'punctuation_blocs', 'expressive_punctuation', 'longest_same_nonpunctuation_char', 'digit_sequences', 'capitalization', 'splitted_into_paragraphs', 'mean_sent_len', 'duplicates', 'mean_word_len']]

In [95]:
prediction_translated1 = model.predict(X_valid_translated)

In [96]:
sklearn.metrics.f1_score(y_valid, prediction_translated1)

0.08734602463605823

In [98]:
sklearn.metrics.roc_auc_score(y_valid, prediction_translated1)

0.5204259300022012

In [99]:
prediction_translated2 = model2.predict(X_valid_translated)

In [100]:
sklearn.metrics.f1_score(y_valid, prediction_translated2)

0.22328767123287674

In [101]:
sklearn.metrics.roc_auc_score(y_valid, prediction_translated2)

0.5451455535989433

Для логистической регрессии roc_ouc на переводе почти такой же, как на оригинале (для обученной на английском модели), f-мера немного хуже.
Для леса обе метрики лучше на переводе, что подтверждает приемлемость использования перевода для данной задачи